In [1]:
import numpy as np
import scipy.optimize
import os
import re
import json
import matplotlib.pyplot as plt

In [2]:
tmp_dir = os.path.join(
    os.environ["USERPROFILE"], "rzgdatashare", "RZG_Data"
)
tail_name = "20181109_SLD670_L4-center_MMF20-22-105_L11_L500_Coherence"
subfolder = "20181109_SLD670_Coherence"
var_format = "{:s}_v_x_{:d}um"
var_list = list(np.arange(4900, 5081, 10))
var_title = "SLD670, L4 (center), MMF20-22-105, L11, L500"
var_label = "Lateral displacement (µm)"

var_folders = [
    os.path.join(tmp_dir, tail_name, var_format.format(subfolder, x))
    for x in var_list
]
for folder in var_folders:
    if not os.path.isdir(folder):
        raise FileNotFoundError(folder)

In [3]:
def get_crop_coords(file_name):
    lines = []
    with open(file_name) as f:
        for line in f:
            if re.match(r"#*", line):
                lines.append(line.strip("# \r\n"))
    header = None
    for line in lines:
        try:
            header = json.loads(line)
        except ValueError as e:
            continue
        if type(header) == dict:
            break
    assert(header is not None and
           "piezo_trace" in header.keys() and
           "cohtrace_coords" in header.keys() and
           "crop_coords" in header.keys())
    unit = header["piezo_trace"]
    cohtrace_coords = header["cohtrace_coords"]
    crop_coords = header["crop_coords"]
    return crop_coords

In [16]:
coherence = []
crop_coords = []
coh_totmax = []
coh_max = []
coh_center = []
totmax = [0, 0, 0]      # [x, y, val]
coh_maxtrace = []

for folder in var_folders:
    crop_coords.append(get_crop_coords(os.path.join(
        folder, os.path.basename(folder) + "_cohtrace.txt"
    )))
crop_coords = np.array(crop_coords)
min_ = np.min(crop_coords, axis=0)
max_ = np.max(crop_coords, axis=0)
target_coords = np.array([[max_[0, 0], max_[0, 1]], [min_[1, 0], min_[1, 1]]])
diff_coords = np.array([target_coords - cc for cc in crop_coords])
for it, d in enumerate(diff_coords):
    if d[1, 0] == 0:
        diff_coords[it, 1, 0] = crop_coords[it, 1, 0]
    if d[1, 1] == 0:
        diff_coords[it, 1, 1] = crop_coords[it, 1, 1]

for it, folder in enumerate(var_folders):
    im = (
        -np.load(os.path.join(folder, os.path.basename(folder) + "_imrec_min.npy"))
    )[diff_coords[it, 0, 0]:diff_coords[it, 1, 0], diff_coords[it, 0, 1]:diff_coords[it, 1, 1]]
    
    _totmax = np.unravel_index(np.argmax(im), im.shape)
    coh_totmax.append(im[_totmax])
    if im[_totmax] > totmax[2]:
        totmax[0], totmax[1] = _totmax[0], _totmax[1]
        totmax[2] = im[_totmax]
    
    xcut, ycut = im.shape[0] // 3, im.shape[1] // 3
    im_max = im[xcut:2 * xcut, ycut:2 * ycut]
    coh_max.append(np.max(im_max))
    
    xcut, ycut = im.shape[0] // 2, im.shape[1] // 2
    im_center = im[xcut - 10:xcut + 11, ycut - 10:ycut + 11]
    coh_center.append(np.max(im_center))
    
for it, folder in enumerate(var_folders):
    im = (
        -np.load(os.path.join(folder, os.path.basename(folder) + "_imrec_min.npy"))
    )[diff_coords[it, 0, 0]:diff_coords[it, 1, 0], diff_coords[it, 0, 1]:diff_coords[it, 1, 1]]
    coherence.append(im)
    coh_maxtrace.append(im[totmax[0], totmax[1]])
    
coherence = np.array(coherence)

In [5]:
%matplotlib
var_list = np.array(var_list) - 5000
plt.plot(var_list, coh_totmax, label="totmax")
plt.plot(var_list, coh_max, label="max")
plt.plot(var_list, coh_center, label="center")
plt.plot(var_list, coh_maxtrace, label="maxtrace")
plt.title(var_title)
plt.xlabel(var_label)
plt.ylabel("Spatial coherence")
plt.legend()
plt.show()

Using matplotlib backend: TkAgg


In [6]:
def stat(data, weight, x, y):
    """data: lateral position, weight: coherence"""
    weight_sum = np.sum(weight)
    mean, std = 0.0, 0.0
    if weight_sum > 0:
        mean = np.sum(data * weight) / weight_sum
        std = np.sqrt(np.sum((data - mean)**2 * weight) / weight_sum)
    if np.isnan(mean) or np.isnan(std):
        print(x, y, "isnan")
    if np.isinf(mean) or np.isinf(std):
        print(x, y, "isinf")
    return mean, std
coherence_mean = np.full(coherence.shape[1:], 0.0)
coherence_std = np.full(coherence.shape[1:], 0.0)
for x in range(coherence.shape[1]):
    for y in range(coherence.shape[2]):
        coherence_mean[x, y], coherence_std[x, y] = stat(var_list, coherence[:, x, y], x, y)

In [7]:
def fit_func(x, A, x0, s, y0):
    return A * np.exp(-(x - x0)**2 / 2 / s**2) + y0
def fit(data, weight, mean, std):
    """data: lateral position, weight: coherence"""""
    weight = np.array(weight) / np.sum(weight)
    min_ = np.min(weight)
    max_ = np.max(weight)
    if np.isclose(std, 0):
        std = 20.0
    if np.any(np.isnan(weight)):
        print("isnan")
    if np.any(np.isinf(weight)):
        print("isinf")
    res = (None, None)
    try:
        res = scipy.optimize.curve_fit(
            fit_func, data, weight,
            p0=(max_ - min_, mean, std, min_)
        )
    except(RuntimeError):
        res = ([np.nan, np.nan, np.nan, np.nan], None)
    return res
coherence_x0 = np.full(coherence.shape[1:], 0.0)
coherence_s = np.full(coherence.shape[1:], 0.0)
for x in range(coherence.shape[1]):
    for y in range(coherence.shape[2]):
        try:
            param, cov = fit(
                var_list, coherence[:, x, y],
                coherence_mean[x, y], coherence_std[x, y]
            )
            coherence_x0[x, y], coherence_s[x, y] = param[1], param[2]
        except(ValueError):
            print(x, y, ValueError)

C:\Users\LaborUser\AppData\Local\Programs\Python\Python37\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [8]:
plt.imshow(coherence_mean)
plt.colorbar()
plt.title("{:s}: coh. mean (µm)".format(var_title))
plt.xlabel("x (px)")
plt.ylabel("y (px)")
plt.show()

In [9]:
plt.imshow(coherence_std)
plt.colorbar()
plt.title("{:s}: coh. std (µm)".format(var_title))
plt.xlabel("x (px)")
plt.ylabel("y (px)")
plt.show()

In [13]:
plt.imshow(coherence_x0)
plt.colorbar()
plt.title("{:s}: coh. Gs. x0 (µm)".format(var_title))
plt.xlabel("x (px)")
plt.ylabel("y (px)")
plt.show()

In [20]:
plt.imshow(coherence_s, vmin=10, vmax=80)
plt.colorbar()
plt.title("{:s}: coh. Gs. s (µm)".format(var_title))
plt.xlabel("x (px)")
plt.ylabel("y (px)")
plt.show()

In [21]:
hist, bins = np.histogram(coherence_s.flatten(), bins=81)
bin_width = (bins[-1] - bins[0]) / (len(bins) - 1)
hist = hist / np.sum(hist) / bin_width
bins = np.array([(bins[i] + bins[i + 1]) / 2 for i in range(len(bins) - 1)])
plt.bar(bins, hist)
def fit_func(x, A, x0, s, y0):
    return A * np.exp(-(x - x0)**2 / 2 / s**2) + y0
p, _ = scipy.optimize.curve_fit(
    fit_func, bins, hist,
    p0=(np.max(hist), bins[len(bins) // 2], bins[int(0.75 * len(bins))] - bins[len(bins) // 2], 0)
)
y = fit_func(np.linspace(bins[0], bins[-1], 200), *p)
plt.plot(np.linspace(bins[0], bins[-1], 200), y, label="coh. length: {:.1f}µm".format(p[1]), color="C1")
plt.title("{:s}: hist. Gs. s".format(var_title))
plt.xlabel("Coherence length (µm)")
plt.ylabel("PDF")
plt.legend()
plt.show()